## Test model, visualize outputs


In [ ]:
#!pip install pytorch-fid

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import os,sys,inspect, random
sys.path.insert(0,"..")

import matplotlib.pyplot as plt
from matplotlib import rc
#rc('font',**{'family':'sans-serif','sans-serif':['Computer Modern Roman']})
#rc('text', usetex=True)

import matplotlib.colors
import cv2
import numpy as np
import string


import pytorch_ssim
import torch
from torch.autograd import Variable

In [ ]:
def visualize(idx, **images):
    """Plot images in one row."""
    
    #norm=plt.Normalize(0,4) # 5 classes including BG
    #map_name = matplotlib.colors.LinearSegmentedColormap.from_list("", ["black", "red","yellow","blue", "green"])
    
    n = len(images)
    plt.figure(figsize=(10, 8))
    for i, (name, image) in enumerate(images.items()):
        plt.subplot(1, n, i + 1)
        plt.xticks([])
        plt.yticks([])
        #plt.title(' '.join(name.split('_')).title())
        plt.imshow(image)
        
    plt.savefig("../outs/compare/{}.png".format(idx), facecolor="white", bbox_inches = 'tight', dpi=600)
    #plt.show()
    
    
def make_dataset(dir):
    images = []
    assert os.path.isdir(dir), '%s is not a valid directory' % dir

    f = dir.split('/')[-1].split('_')[-1]
    #print (dir, f)
    dirs= os.listdir(dir)
    for img in dirs:

        path = os.path.join(dir, img)
        #print(path)
        images.append(path)
    return images

def read_image(path):
    image = cv2.imread(path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image = cv2.resize(image, (320, 512))
    return image

In [ ]:
os.makedirs('../outs/compare',exist_ok=True)

In [ ]:
# path to tryons

# Same
# cloth = "../results/same/aligned/TFM/test_pairs/cloth"
# person = "../results/same/aligned/TFM/test_pairs/person"
# tryon1 = "../results/same/aligned_unet/TFM/test_pairs/tryon"
# tryon2 = "../results/same/aligned/TFM/test_pairs/tryon"

# Random
cloth = "../results/random/aligned/TFM/test_pairs/cloth"
person = "../results/random/aligned/TFM/test_pairs/person"
tryon1 = "../results/random/aligned_unet/TFM/test_pairs/tryon"
tryon2 = "../results/random/aligned/TFM/test_pairs/tryon"

In [ ]:
cloths = sorted(make_dataset(cloth))
persons = sorted(make_dataset(person))
tryon1p = sorted(make_dataset(tryon1))
tryon2p = sorted(make_dataset(tryon2))

In [ ]:
len(tryon2p)

### Visualize different algo output

In [ ]:
# slc = 934
# i = 0
# for c, p, a1, a2 in zip(cloths[:slc], persons[:slc], tryon1p[:slc], 
#                      tryon2p[:slc]):
    
#     # visualize(i,
#     #           unet=read_image(a1), 
#     #           resunet = read_image(a2))
#     visualize(i, 
#               person=read_image(p),
#               cloth=read_image(c),
#               unet=read_image(a1), 
#               resunet = read_image(a2))
#     i+=1

### Vis subset 

In [ ]:
# nums = np.array([4,5,22,45,87,123,10,52,95,
#                  99,323,586,149,188,787,806,829,323,929,325,241,193,276,294,306])

# nums = nums[:12]

nums = [4,5,22,45,87,123,10,52,95,
                 99,323,586,149,188,787,806,829,325,241,306,276,294]

# Plot these
nums = [123, 829, 10, 99, 276, 95, 52, 294] #random.sample(nums, 8)

rows = int(len(nums) / 2)
cols = 8
fig, axarr = plt.subplots(rows, cols, figsize=(22, 18), constrained_layout=True)

v = 0
for r in range(rows):
    rp1=read_image(persons[nums[v+r]])
    tc1=read_image(cloths[nums[v+r]])
    a1=read_image(tryon1p[nums[v+r]])
    o1=read_image(tryon2p[nums[v+r]])
    
    rp2=read_image(persons[nums[v+r+1]])
    tc2=read_image(cloths[nums[v+r+1]])
    a2=read_image(tryon1p[nums[v+r+1]])
    o2=read_image(tryon2p[nums[v+r+1]])
    
    # rp3=read_image(persons[nums[v+r+2]])
    # tc3=read_image(cloths[nums[v+r+2]])
    # a3=read_image(tryon1p[nums[v+r+2]])
    # o3=read_image(tryon2p[nums[v+r+2]])
    
    v+=1
    #images = [rp1, tc1, a1, o1, rp2, tc2, a2, o2, rp3, tc3, a3, o3]
    images = [rp1, tc1, a1, o1, rp2, tc2, a2, o2]
    
    captions = ["Reference \n Person", "Target \n Clothes", "M3D-VTON", "Ours", 
                "Reference \n Person", "Target \n Clothes", "M3D-VTON", "Ours", 
                "Reference \n Person", "Target \n Clothes", "M3D-VTON", "Ours"]
    
    for c in range(cols):
        axarr[r, c].imshow(images[c], cmap='gray')
        axarr[r, c].axis("off")
        axarr[r, c].set_aspect('equal')
        if r==0:
            axarr[r, c].set_title(captions[c], fontsize=25)

plt.savefig("../outs/visualization_tryons_small.pdf", facecolor="white", bbox_inches = 'tight', dpi=100)

### All

In [ ]:
nums = np.array([4,5,22,45,87,123,10,52,95,
                 99,323,586,149,188,787,806,829,193,929,325,241,306,276,294])

nums = np.array([4,5,22,45,87,123,10,52,95,
                 99,323,149,188,787,806,829,325,241,306,276,294])

#nums = np.array([99,323,149,188,787,806,829,325,241,306,276,294])


rows = int(7)
cols = 12
fig, axarr = plt.subplots(rows, cols, figsize=(33, 31), constrained_layout=True)
#fig, axarr = plt.subplots(rows, cols, figsize=(33, 15), constrained_layout=True)
alphabet_string = string.ascii_lowercase
alphabet_list = list(alphabet_string)

v = 0
for r in range(rows):
    rp1=read_image(persons[nums[v+r]])
    w,h = (1,50)
    fs = 2
    color = (255,255,255)
    font = cv2.FONT_HERSHEY_SIMPLEX #FONT_HERSHEY_DUPLEX  #press tab for different operations
    cv2.putText(rp1, str(alphabet_list[v+r]), (w,h), font, fs, color, 3, cv2.LINE_AA)
    
    tc1=read_image(cloths[nums[v+r]])
    a1=read_image(tryon1p[nums[v+r]])
    o1=read_image(tryon2p[nums[v+r]])
    
    rp2=read_image(persons[nums[v+r+1]])
    cv2.putText(rp2, str(alphabet_list[v+r+1]), (w,h), font, fs, color, 3, cv2.LINE_AA)
    tc2=read_image(cloths[nums[v+r+1]])
    a2=read_image(tryon1p[nums[v+r+1]])
    o2=read_image(tryon2p[nums[v+r+1]])
    
    rp3=read_image(persons[nums[v+r+2]])
    cv2.putText(rp3, str(alphabet_list[v+r+2]), (w,h), font, fs, color, 3, cv2.LINE_AA)
    tc3=read_image(cloths[nums[v+r+2]])
    a3=read_image(tryon1p[nums[v+r+2]])
    o3=read_image(tryon2p[nums[v+r+2]])
    
    v+=2
    images = [rp1, tc1, a1, o1, rp2, tc2, a2, o2, rp3, tc3, a3, o3]
    
    captions = ["Reference \n Person", "Target \n Clothes", "M3D-VTON", "Ours", 
                "Reference \n Person", "Target \n Clothes", "M3D-VTON", "Ours", 
                "Reference \n Person", "Target \n Clothes", "M3D-VTON", "Ours"]
    
    for c in range(cols):
        axarr[r, c].imshow(images[c], cmap='gray')
        axarr[r, c].axis("off")
        axarr[r, c].set_aspect('equal')
        if r==0:
            axarr[r, c].set_title(captions[c], fontsize=35)

plt.savefig("../outs/visualization_tryons_all.png", facecolor="white", bbox_inches = 'tight', dpi=100)

### Visualize full input, 2D, 3D outputs

In [ ]:

nums = np.array([4,5,22,45,87,123,10,52,95,
                 99,323,149,188,787,806,829,325,241,306,276,294])

In [ ]:
len(nums)

In [ ]:
persons[nums[4]], persons[nums[8]], persons[nums[9]], persons[nums[10]]

In [ ]:
def read_image_(path):
    image = cv2.imread(path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    #image = cv2.resize(image, (320, 512))
    image = center_crop(image, (320, 512))
    #image = cv2.resize(image, (320, 512))
    image = scale_image(image, factor=1)
    return image


# Code taken from https://gist.github.com/Nannigalaxy/35dd1d0722f29672e68b700bc5d44767
def center_crop(img, dim):
	"""Returns center cropped image
	Args:
	img: image to be center cropped
	dim: dimensions (width, height) to be cropped
	"""
	width, height = img.shape[1], img.shape[0]

	# process crop width and height for max available dimension
	crop_width = dim[0] if dim[0]<img.shape[1] else img.shape[1]
	crop_height = dim[1] if dim[1]<img.shape[0] else img.shape[0] 
	mid_x, mid_y = int(width/2), int(height/2)
	cw2, ch2 = int(crop_width/2), int(crop_height/2) 
	crop_img = img[mid_y-ch2:mid_y+ch2, mid_x-cw2:mid_x+cw2]
	return crop_img

def scale_image(img, factor=1):
	"""Returns resize image by scale factor.
	This helps to retain resolution ratio while resizing.
	Args:
	img: image to be scaled
	factor: scale factor to resize
	"""
	return cv2.resize(img,(int(img.shape[1]*factor), int(img.shape[0]*factor)))


# def image_resize(path, width = None, height = None, inter = cv2.INTER_AREA):
    
#     image = cv2.imread(path)
#     image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    
#     # initialize the dimensions of the image to be resized and
#     # grab the image size
#     dim = None
#     (h, w) = image.shape[:2]

#     # if both the width and height are None, then return the
#     # original image
#     if width is None and height is None:
#         return image

#     # check to see if the width is None
#     if width is None:
#         # calculate the ratio of the height and construct the
#         # dimensions
#         r = height / float(h)
#         dim = (int(w * r), height)

#     # otherwise, the height is None
#     else:
#         # calculate the ratio of the width and construct the
#         # dimensions
#         r = width / float(w)
#         dim = (width, int(h * r))

#     # resize the image
#     resized = cv2.resize(image, dim, interpolation = inter)

#     # return the resized image
#     return resized


rp1=read_image(persons[nums[4]])
w,h = (1,50)
fs = 2
color = (255,255,255)
font = cv2.FONT_HERSHEY_SIMPLEX #FONT_HERSHEY_DUPLEX  #press tab for different operations
cv2.putText(rp1, str('a'), (w,h), font, fs, color, 3, cv2.LINE_AA)
tc1=read_image(cloths[nums[4]])
a1=read_image(tryon1p[nums[4]])
o1=read_image(tryon2p[nums[4]])

rp2=read_image(persons[nums[8]])
cv2.putText(rp2, str('b'), (w,h), font, fs, color, 3, cv2.LINE_AA)
tc2=read_image(cloths[nums[8]])
a2=read_image(tryon1p[nums[8]])
o2=read_image(tryon2p[nums[8]])

rp3=read_image(persons[nums[9]])
cv2.putText(rp3, str('c'), (w,h), font, fs, color, 3, cv2.LINE_AA)
tc3=read_image(cloths[nums[9]])
a3=read_image(tryon1p[nums[9]])
o3=read_image(tryon2p[nums[9]])

rp4=read_image(persons[nums[10]])
cv2.putText(rp4, str('d'), (w,h), font, fs, color, 3, cv2.LINE_AA)
tc4=read_image(cloths[nums[10]])
a4=read_image(tryon1p[nums[10]])
o4=read_image(tryon2p[nums[10]])


# a13d = image_resize("../assets/unet1.png")
# a13dd = image_resize("../assets/unet2.png")

pcu1 = read_image_("../assets/unet1.png")
pcu2 = read_image_("../assets/unet2.png")
pcu3 = read_image_("../assets/unet3.png")
pcu4 = read_image_("../assets/unet4.png")
pcur1 = read_image_("../assets/resunet1.png")
pcur2 = read_image_("../assets/resunet2.png")
pcur3 = read_image_("../assets/resunet3.png")
pcur4 = read_image_("../assets/resunet4.png")

In [ ]:

rows = int(4)
cols = 6
fig, axarr = plt.subplots(rows, cols, figsize=(20, 22), constrained_layout=True)

# [rp1, tc1, a1, o1, pcu2, pcur2],
# [rp3, tc3, a3, o3, pcu3, pcur3],

images = [[rp1, tc1, a1, o1, pcu2, pcur2],
          [rp2, tc2, a2, o2, pcu1, pcur1],
          [rp3, tc3, a3, o3, pcu3, pcur3],
          [rp4, tc4, a4, o4, pcu4, pcur4]]
    
captions = ["Reference \n Person", "Target \n Clothes", "M3D-VTON (2D)", "Ours (2D)", "M3D-VTON \n (3D Mesh)", "Ours \n (3D Mesh)",]
v=0
for r in range(rows):
    img = images[r]
    for c in range(cols):
        axarr[r, c].imshow(img[c], cmap='gray')
        axarr[r, c].axis("off")
        axarr[r, c].set_aspect('equal')
        if r==0:
            axarr[r, c].set_title(captions[c], fontsize=25)
            
    v+=1

plt.savefig("../outs/vis_3d.png", facecolor="white", bbox_inches = 'tight', dpi=100)

### Compute FID scores

In [ ]:
!python -m pytorch_fid "/home/hz/m3d-vton/results/same/aligned_unet/TFM/test_pairs/person/" "/home/hz/m3d-vton/results/same/aligned_unet/TFM/test_pairs/person/"

#### Baseline

In [ ]:
!python -m pytorch_fid "/home/hz/m3d-vton/results/same/aligned_unet/TFM/test_pairs/person/" "/home/hz/m3d-vton/results/same/aligned_unet/TFM/test_pairs/tryon"

#### Ours

In [ ]:
!python -m pytorch_fid "/home/hz/m3d-vton/results/same/aligned_unet/TFM/test_pairs/person/" "/home/hz/m3d-vton/results/same/aligned/TFM/test_pairs/tryon"

### Compute SSIM

In [ ]:
scores = []

gt = "/home/hz/m3d-vton/results/same/aligned_unet/TFM/test_pairs/person/"
a1 = "/home/hz/m3d-vton/results/same/aligned_unet/TFM/test_pairs/tryon"
a2 = "/home/hz/m3d-vton/results/same/aligned/TFM/test_pairs/tryon"

gtp = sorted(make_dataset(gt))
a1p = sorted(make_dataset(a1))
a2p = sorted(make_dataset(a2))

for x,y in zip(gtp, a2p):
    img1 = read_image(x) / 255.0
    img2 = read_image(y) / 255.0
    
    img1 = np.expand_dims(img1, axis=0)
    img2 = np.expand_dims(img2, axis=0)
    
    if torch.cuda.is_available():
        img1 = torch.from_numpy(img1)
        img2 = torch.from_numpy(img2)
        
        #print(img1.shape, img2.shape)
        score = pytorch_ssim.ssim(img1, img2)
        score = score.item()
        #print(score)
        scores.append(score)

In [ ]:
score = np.array(scores).mean()
print("SSIM score: {:.4f}".format(score))

| Method | FID (↓) | SSIM (↑)|
|--------|------|-----|
| VITON (CVPR, 2018) | 28.43 | 0.8807 |
| CP-VTON (ECCV, 2018) | 20.05 | 0.8503 |
| CP-VTON+ (CVPRW, 2020) | 23.18 | 0.8782 |
| ACGPN (CVPR, 2020) | 20.19 | 0.8924 |
| M3D-VTON (ICCV, 2021) | 19.875 | 0.9725 |
| Ours | **15.162** | **0.9814** |
| Ours | **14.591** | **0.9817** |